In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Veri Seti Kısa Hikayesi;
#Bu veri seti Pima Indian ırkından en genç 21 yaşında olan kadınların tıbbi kayıtlarına göre diyabet hastası olup olmadığını ve hastalığa sebep olan etkenleri(bagimsiz degiskenleri) de bir arada bulundurmaktadır.
## Değişkenler;
 Pregnancies: Hamile kalma sayısı.
 
 Glucose: Glikoz.
 
 BloodPressure: Kan basıncı.
 
 SkinThickness : Deri kalınlığı.
 
 Insulin: İnsülin.
 
 BMI: Beden kitle indeksi.
 
 DiabetesPedigreeFunction: Soyumuzdaki kişilere göre diyabet olma ihtimalimizi hesaplayan bir fonksiyon.
 
 Age: Yaş.
 
 Outcome: Kişinin diyabet olup olmadığı bilgisi.

In [ ]:
# Kutuphane Import Islemleri
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
pd.set_option("display.max_columns", None)

In [ ]:
# Veri Setini Okutmak
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
# Veri Setinin İlk Bes Gozlemine Erismek
df.head()

Tanımlayıcı İstatistik

In [ ]:
df.describe().T

Pregnancies ve Outcome dışındaki değişkenlerde yer alan sıfırlar eksik gözlemi ifade etmektedir. Bundan dolayı ilgili değişkenleri seçip sıfır olan gözlem değerlerini boş olarak atayalım. 

In [ ]:
zero_columns = [i for i in df.columns if (df[i].min() == 0 and i not in ["Pregnancies", "Outcome"])]
for i in zero_columns:
    df[[i]] = df[[i]].replace(0, np.NaN)



In [ ]:
df.isnull().sum()

# Veri Ön İşleme

Eksik değerlerin Outcome sınıf kırılımına göre doldurulması

In [ ]:
for i in zero_columns:
    df[i] = df[i].fillna(df.groupby("Outcome")[i].transform("median"))

In [ ]:
# Veri setinde eksik gozlem degeri kalip kalmadigini kontrol edelim
df.isnull().values.any()

In [ ]:
# Aykiri Deger Analizi
# Boxplot Yöntemi
# Oncelikle Outcome degiskeni haricindekileri secelim
num_cols = [col for col in df.columns if col != "Outcome"]

# Belirlenen thresholds degerlerine gore alt ve ust limitleri hesaplaması icin fonksiyon tanimliyalim.
def outlier_thresholds(dataframe, col_name):
    quartile1 = dataframe[col_name].quantile(0.10)
    quartile3 = dataframe[col_name].quantile(0.90)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

# Hesaplanan alt limitin altinda veya ust limitin ustunde bir deger varsa True yoksa False dondurecek fonksiyonu tanimlayalim
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

# num_cols listesinde bulunan degisken isimlerine göre aykiri deger olup olmadigini gorelim
for col in num_cols:
    print(col, ":", check_outlier(df, col))

In [ ]:
# Insulin, SkinThickness ve DiabetesPedigreeFunction degiskenlerinde bulunan aykiri degerleri alt veya ust limite gore baskilayalim.
# Bunun icin bir fonksiyon yazalim
def replace_with_thresholds(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if low_limit > 0:
        dataframe.loc[(dataframe[col_name] < low_limit), col_name] = low_limit
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit
    else:
        dataframe.loc[(dataframe[col_name] > up_limit), col_name] = up_limit

replace_with_thresholds(df, "Insulin")
replace_with_thresholds(df, "SkinThickness")
replace_with_thresholds(df, "DiabetesPedigreeFunction")



In [ ]:
# Aykiri deger olup olmadigini kontrol edelim
for col in num_cols:
    print(col, ":", check_outlier(df, col))

In [ ]:
# Sayisal Degisken Analizi
# Histogram grafigi
for i in num_cols:
    sns.histplot(x=df[i], data=df)
    plt.title("distribution")
    plt.xlabel(i)
    plt.show()
    

In [ ]:
# TARGET degiskenine gore sayısal degiskenlerin analizi
for col in num_cols:
    print(df.groupby("Outcome").agg({col: ["min", "mean", "median", "std", "max"]}))
    print("-"*30)

In [ ]:
# Outcome siniflarina gore sayisal degiskenlerin medyan degerleri arasinda farklilik olup olmadigini inceleyelim.
# Oncelikle normallik testi yapilmalidir.
for col in num_cols:
    for i in df["Outcome"].unique():
        test_stast, pvalue = stats.shapiro(df.loc[df["Outcome"] == i, col])
        print(f"Numeric variable: {col} Class: {i} pvalue: {pvalue}")
    

In [ ]:
# Normallik saglanmadigi icin Mann-Whitney U Testi Yapalim
for col in num_cols:
    test_stats, pvalue = stats.mannwhitneyu(df.loc[df["Outcome"] == df["Outcome"].unique()[0], col],
                                           df.loc[df["Outcome"] == df["Outcome"].unique()[1], col])
    print(f"Numeric variable: {col} pvalue: {pvalue}")

In [ ]:
# Medyan degerleri farklilik gostermektedir. Tekrar Outcome degiskenin sinif kiriliminda diger degiskenlerin medyan degerlerine bakalim
for col in num_cols:
    print(df.groupby("Outcome").agg({col: "median"}))
    print("-"*30)

Glukoz ve insulin degerlerine baktigimiz zaman diabet hastasi olan kisilerin medyan degeri diabet hastası olmayanlara gore hem yuksek hem de istatistiksel olarak
farklilik anlamlidir. Diger degiskenler icinde medyan degerleri arasinda bir farkliligin istatistiksel olarak anlamli oldugunu soyleyebiliriz. Ama goze carpan bir farkliligin Glukoz ve Insulin degiskenleri icin gozlemlemekteyiz. Yeni degisken turetmek icin bu degiskenler goz onunde bulundurulabilir.

In [ ]:
# Degisken Turetelim. Olasi tum degiskenleri turetelim
df["PREG_AGE"] = df["Pregnancies"] * df["Age"]

df["Glucose_BMI"] = df["Glucose"] * df["BMI"]

df["Insulin_Glucose"] = df["Insulin"] * df["Glucose"]

df["Insulin_BMI"] = df["Insulin"] * df["BMI"]

df["INSULİN_AGE"] = df["Insulin"] * df["Age"]

# Insulin 16 ile 166 arasi normal diger durumlar anormal
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"


df = df.assign(NewInsulinScore=df.apply(set_insulin, axis=1))


In [ ]:
df.head()

In [ ]:
# Label Encoding
binary_cols = [col for col in df.columns if df[col].dtypes == "O" and df[col].nunique() == 2]
def label_encoder(dataframe, binary_col):
    labelencoder = preprocessing.LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

for col in binary_cols:
    df = label_encoder(df, col)


In [ ]:
df.head()

In [ ]:
y = df["Outcome"]
x = df.drop("Outcome", axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

print(f"{x_train.shape}, {x_test.shape}, {y_train.shape}, {y_test.shape}")

In [ ]:
# Model Kurmak
cls = DecisionTreeClassifier()

model = cls.fit(x_train, y_train)
proba = model.predict_proba(x_test)[:, 1]
pred = model.predict(x_test)
print(f"AUC: {roc_auc_score(y_test, proba)}\nACCURACY: {accuracy_score(y_test, pred)}\nPRESICION: {precision_score(y_test, pred)}\nRECALL: {recall_score(y_test, pred)}")

In [ ]:
# Model Get Params
model.get_params()

In [ ]:
# Model Tunning
cart_model = DecisionTreeClassifier(random_state=17)

# arama yapılacak hiperparametre setleri
cart_params = {'max_depth': range(1, 11),
               "min_samples_split": range(1, 16)}


cart_cv = GridSearchCV(cart_model, cart_params, cv=5, n_jobs=-1, verbose=True)
cart_cv.fit(x_train, y_train)

In [ ]:
# Final Model
cart_tuned = DecisionTreeClassifier(**cart_cv.best_params_).fit(x_train, y_train)
y_pred = cart_tuned.predict(x_test)
y_prob = cart_tuned.predict_proba(x_test)[:, 1]
print(f"AUC: {roc_auc_score(y_test, y_prob)}\nACCURACY: {accuracy_score(y_test, y_pred)}\nPRESICION: {precision_score(y_test, y_pred)}\nRECALL: {recall_score(y_test, y_pred)}")